In [ ]:
import getpass
import requests
import pandas as pd
import xlsxwriter

In [ ]:
username=input("NTS username: ")
password = getpass.getpass("NTS password: ")

In [ ]:
url = "https://terminologieserver.nl/auth/realms/nictiz/protocol/openid-connect/token"

payload='username='+username+'&password='+password+'&grant_type=password&client_id=cli_client'
headers = { 'Content-Type': 'application/x-www-form-urlencoded' }

response = requests.request("POST", url, headers=headers, data=payload)

token = response.json()
access_token = token["access_token"]

#print(access_token)

In [ ]:
url = "https://terminologieserver.nl/fhir/ValueSet/?title=tnmversiecodelijst"

payload={}
headers = { 'Authorization': 'Bearer ' + access_token }

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

In [ ]:
data = response.json()

print (data["entry"][0]["resource"]["id"])

In [ ]:
url = "https://terminologieserver.nl/fhir/ValueSet/" + data["entry"][0]["resource"]["id"]

payload={}
headers = { 'Authorization': 'Bearer ' + access_token }

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

In [ ]:
data = response.json()

concepts = []
for include in data["compose"]["include"]:
    for concept in include["concept"]:
        concept["system"] = include["system"]
        concepts.append(concept)
        print(concept)

In [ ]:
df = pd.DataFrame(concepts)
df = df[['system','code', 'display']]
print("Concepts in Valueset:", df.size)
df.head(20)

In [ ]:
writer = pd.ExcelWriter('valueset.xlsx')
df.to_excel(writer)
writer.save()